In [ ]:
import h5py
from nexusformat.nexus import *
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np

from utils import file_searcher, alloy_inserter

In [ ]:
sample_id, number = file_searcher("./growths/processes/")
print(sample_id)
print("Total number of files:", len(sample_id))

In [ ]:
file_path = "nexus_files/hm4721IGA.nxs"

## NeXus Structure and Tree

In [ ]:
# Open your HDF5 file
with h5py.File(file_path, "r") as f:
    def print_h5_structure(name, obj):
        print(name, obj)
    
    f.visititems(print_h5_structure)  # Prints dataset structure

In [ ]:
test = nxload(file_path)
print(test.tree)

## Sample Visualization

In [ ]:
with h5py.File(file_path, 'r') as f:
    sample = f['/entry/sample']
    layer_keys = sorted(
        [k for k in sample.keys() if k.startswith("layer")],
        key=lambda x: int(x.replace("layer", ""))
    )

    layers = []
    for key in layer_keys:
        layer = sample[key]
        thickness = float(layer.get('thickness', [0])[()]) / 10.0  # Å to nm
        material = layer.get('chemical_formula', b'').asstr()[()] if 'chemical_formula' in layer else 'unknown'
        alloy = layer.get('alloy_fraction', [0])[()]
        if len(material) == 6:
            new_material = alloy_inserter(material, alloy)
        else:
            new_material = material
        layers.append((material, thickness, new_material))

    substrate = f['/entry/sample/substrate']
    orientation = substrate.get('crystal_orientation', b'').asstr()[()] if 'crystal_orientation' in substrate else 'unknown'

layers.insert(0, ("Substrate", 0, f"GaAs {orientation} substrate"))

# Draw Equal-Height Stack 
fig, ax = plt.subplots(figsize=(8, 12))
block_height = 1  
y = 0
total_blocks = len(layers)

color_map = {
    "GaAs": "lightpink",
    "InAs": "red",
    "InAlAs": "skyblue",
    "InGaAs": "gold",
    "AlGaAs": "deepskyblue",
    "Si": "lightgreen",
    "C": "lightgrey",
    "Substrate": "grey"
}

for i, (material, thickness, new_material) in enumerate(reversed(layers)):
    color = color_map.get(material, "black")
    rect = patches.Rectangle((0, y), 1, block_height, facecolor=color, edgecolor='k')
    ax.add_patch(rect)

    if "Substrate" in material:
        label = f"{new_material}"
    else:
        label = f"{new_material} {round(thickness, 1)} nm"

    ax.text(0.5, y + block_height / 2, label,
            ha='center', va='center', fontsize=17)

    y += block_height

ax.set_xlim(0, 1)
ax.set_ylim(0, total_blocks)
ax.invert_yaxis()
ax.axis('off')
#plt.title("Sample Visualization", pad=20)
plt.tight_layout()
plt.show()

## Growth Parameters vs Layers

In [ ]:
variables = ["alloy_fraction", "thickness", "growth_rate", "rotational_frequency", "growth_temperature"]  # Variables to plot

# Read all layers 
with h5py.File(file_path, 'r') as f:
    layer_names = []
    materials = []
    data = {var: [] for var in variables}  # dictionary to store variable values

    for layer_name in f['/entry/sample'].keys():
        if 'layer' in layer_name:
            group = f['/entry/sample'][layer_name]
            layer_names.append(layer_name)
            materials.append(group['chemical_formula'][()].decode())
            for var in variables:
                if var in group:
                    data[var].append(group[var][()])
                else:
                    data[var].append(None)  # handle missing variables

# Plot each variable 
for var in variables:
    y_values = data[var]

    fig, ax = plt.subplots(figsize=(14, 8))
    x = range(len(layer_names))

    bars = ax.bar(x, y_values, color='lightgrey', edgecolor='black', linewidth=0.7)

    ax.set_xlabel('Layer Number', fontsize=20)
    ax.set_ylabel(var.replace('_', ' ').capitalize(), fontsize=20)
    #ax.set_title(f'{var.replace("_", " ").capitalize()} by Layer', fontsize=12)
    tick_labels = [str(i + 1) if (i + 1) % 5 == 0 or i == 0 else '' for i in x]
    ax.set_xticks(x)
    ax.set_xticklabels(tick_labels)
    ax.tick_params(axis='both', labelsize=18)

    secax_x = ax.secondary_xaxis('top')
    secax_x.set_xticks(x)
    secax_x.tick_params(top=True, labeltop=False)
    secax_y = ax.secondary_yaxis('right')
    secax_y.tick_params(right=True, labelright=False)


    # Auto-scale Y based on data
    if all(value is not None for value in y_values):
        y_max = max(y_values) * 1.2
        ax.set_ylim(0, y_max if y_max > 0 else 1)
    else:
        ax.set_ylim(0, 1)  # fallback

    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_linewidth(0.7)
    ax.spines['left'].set_linewidth(0.7)
    ax.grid(axis='y', linestyle='--', alpha=0.3, linewidth=0.5)

    # Annotate each bar with material
    for bar, material in zip(bars, materials):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + 0.01 * max(y_values),
                material, ha='center', va='bottom', fontsize=18, rotation=90)

    plt.tight_layout()
    plt.show()

## Growth Parameters vs Cumulative Thickness

In [ ]:
variables = ["thickness", "alloy_fraction", "growth_rate", "rotational_frequency", "growth_temperature"]
units = ["(nm)", "", "(nm/s)", "(rpm)", "(°C)"]

# --- Read all layers ---
with h5py.File(file_path, 'r') as f:
    layer_names = []
    materials = []
    growth_rate = []
    data = {var: [] for var in variables}

    for layer_name in f['/entry/sample'].keys():
        if 'layer' in layer_name:
            group = f['/entry/sample'][layer_name]
            layer_names.append(layer_name)
            materials.append(group['chemical_formula'][()].decode())
            growth_rate.append(group['growth_rate'][()] * 0.1)
            for var in variables:
                if var in group:
                    data[var].append(group[var][()])
                else:
                    data[var].append(None)

# Convert to nm
thickness_nm = [t * 0.1 if t is not None else 0 for t in data['thickness']]

# Compute bar left edges (start positions)
left_edges = np.cumsum([0] + thickness_nm[:-1])

# Plot each variable 
for var, unit in zip(variables, units):

    if var == "thickness":
        continue
    
    if var == "growth_rate":
        y_values = np.array(data[var]) * 0.1
    else:
        y_values = data[var]

    fig, ax = plt.subplots(figsize=(14, 8))

    # Plot bars with variable widths
    for left, width, height, mat in zip(left_edges, thickness_nm, y_values, materials):
        min_visible_width = 10  # nm threshold

        if height is not None:
            ax.bar(left, height, width=width, align='edge',
                   color='lightgrey', edgecolor='black', linewidth=0.7)
            if width > min_visible_width:
                ax.text(left + width/2, height + 0.01 * max(y_values), mat,
                        ha='center', va='bottom', fontsize=18, rotation=90)
            
    ax.set_xlabel('Thickness (nm)', fontsize=20)
    ax.set_ylabel(var.replace('_', ' ').capitalize() + f" {unit}", fontsize=20)
    #ax.set_title(f'{var.replace("_", " ").capitalize()} by Cumulative Thickness', fontsize=12)

    ax.tick_params(axis='both', labelsize=18)
    # Add a reflected axis 
    secax_x = ax.secondary_xaxis('top')
    secax_x.tick_params(top=True, labeltop=False)

    secax_y = ax.secondary_yaxis('right')
    secax_y.tick_params(right=True, labelright=False)

    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_linewidth(0.7)
    ax.spines['left'].set_linewidth(0.7)
    ax.grid(axis='y', linestyle='--', alpha=0.3, linewidth=0.5)

    plt.tight_layout()
    plt.subplots_adjust(top=0.9)  
    plt.show()